In [2]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
os.chdir(current_dir)

In [4]:
import pandas as pd

# Load the pickled DataFrame
copy = pd.read_pickle("pos_dataframe.pkl")


In [5]:
copy

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group                                                ADJ  \
0    high       doc  first criminal other little more dead opposite...   
1    high       doc  statutory minor sexual female old good female ...   
2    high  handbook  true wonderful dedicated young young local loc...   
3    high       cbc  peel regional public peel regional big crazy c...   
4  medium  handbook  lucky crazy slow steady ordinary last last int...   
5  medium  handbook  low last simple homosexual young correct sound...   
6  medium  handbook  more impossible nice ordinary most sick other ...   
7    high  articles  numerous numerous horrible terrible personal f...   
8    high  articles  nice only right almighty holy true holy other ...   
9     low  handbook  fourth fourth fourth fifth sixth angry alone a...   

                                                 ADP  \
0  of of with on down up with of off on for besid...   
1                       on for on on around of to at   
2  up in with of as of outside in to in of out of...   
3  about with of on for by around on to on with f...   
4  up on like out of to about to of of off for of...   
5  from in of against in on for to for for for of...   
6  for than in up up in out of in as of on in out...   
7  at of of in into with into into like at with o...   
8  in of with with about through through of by of...   
9  to in at for in of at for on during during out...   

                                                 ADV  ...  \
0  very together away ever so just so right sure ...  ...   
1                        then someday very at around  ...   
2  certainly again more generally more more more ...  ...   
3  aka forward maybe here just in back there here...  ...   
4  billy up very just just very so thoroughly jus...  ...   
5  strictly constantly constantly right always aw...  ...   
6  almost far that even increasingly back further...  ...   
7  just everywhere usually just there probably us...  ...   
8  as absolutely so so now once again still very ...  ...   
9  really just just probably probably too so just...  ...   

                                                 NUM  \
0     five twenty four four twelve forty five twenty   
1                                           four one   
2  two one five twelve thirteen one twenty thirty...   
3                          seventeen one one one two   
4  f

In [9]:
import pandas as pd

df = copy

# Extract rows for the test set
test_df = df[df['name'].isin(['ridgeway', 'wayne', 'carl'])]

# Extract the rest of the data for the training set
train_df = df[~df['name'].isin(['ridgeway', 'wayne', 'carl'])]

# Now, 'train_df' is your training set and 'test_df' is your test set


In [10]:
test_df

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group                                                ADJ  \
0    high       doc  first criminal other little more dead opposite...   
1    high       doc  statutory minor sexual female old good female ...   
9     low  handbook  fourth fourth fourth fifth sixth angry alone a...   

                                                 ADP  \
0  of of with on down up with of off on for besid...   
1                       on for on on around of to at   
9  to in at for in of at for on during during out...   

                                                 ADV  ...  \
0  very together away ever so just so right sure ...  ...   
1                        then someday very at around  ...   
9  really just just probably probably too so just...  ...   

                                                 NUM  \
0     five twenty four four twelve forty five twenty   
1                                           four one   
9  one one one one fourteen fifteen two two one o...   

                                                PART  \
0  to na to to nt to to to to to to not to nt to ...   
1                                       not to na to   
9  nt to to to to not to nt nt to to to nt to nt ...   

                                                PRON  \
0  my i my i his he he he me you my you he i his ...   
1                 i i a she its me it my i i i who i   
9  i i her i her what i her her i her me she it i...   

                                               PROPN PUNCT  \
0     sunday i d d sunday january john lennon nelson         
1                                                            
9  shed i d i d i d tan tan jack d d d d d i d d ...   0 0   

                                               SCONJ SYM  \
0  that while if that as that when that that so t...       
1                   for whether that when that where       
9  that how if where that how that when if so whe...       

                                                VERB X SPACE  
0  trying put getting gon kill knew head join lov...          
1  had arrested aggravated be re return gon have m m          
9  think had kill wanted stop wanna do have wante...          

[3 rows x 26 columns]

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df is your training DataFrame
# Assuming test_df is your testing DataFrame

# Extract features and target variable from training data
X_train = train_df[['AUX', 'NUM', 'ADV']]  # Columns for features
y_train = train_df['score']                # Target variable

# Extract features, target variable, and names from testing data
X_test = test_df[['AUX', 'NUM', 'ADV']]
y_test = test_df['score']
names_test = test_df['name']

# Define the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM'),
        ('vect_adv', CountVectorizer(), 'ADV')
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR())
])

# Define the parameter grid to be searched
param_grid = {
    'svr__C': [0.1, 1, 10, 100],            # Example values
    'svr__kernel': ['linear', 'rbf'],       # Example values
    'svr__gamma': ['scale', 'auto']         # Example values
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=7, scoring='neg_mean_absolute_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best Mean Absolute Error:", -grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print predicted, actual values, and names
for actual, predicted, name in zip(y_test, y_pred, names_test):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


# Now you can use grid_search.best_estimator_ as your best performing model


Fitting 7 folds for each of 16 candidates, totalling 112 fits
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, sv

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df is your training DataFrame
# Assuming test_df is your testing DataFrame

# Extract features and target variable from training data
X_train = train_df[['AUX', 'NUM', 'ADV']]  # Columns for features
y_train = train_df['score']                # Target variable

# Extract features, target variable, and names from testing data
X_test = test_df[['AUX', 'NUM', 'ADV']]
y_test = test_df['score']
names_test = test_df['name']

# Define the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM'),
        ('vect_adv', CountVectorizer(), 'ADV')
    ]
)

# Define the pipeline with the column transformer and Lasso
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('lasso', Lasso())
])

# Define the parameter grid to be searched
param_grid = {
    'lasso__alpha': [0.01, 0.1, 1, 10, 100]  # Example values
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=7, scoring='neg_mean_absolute_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best Mean Absolute Error:", -grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print predicted, actual values, and names
for actual, predicted, name in zip(y_test, y_pred, names_test):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Fitting 7 folds for each of 5 candidates, totalling 35 fits
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ..................................lasso__alpha=0.01; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.348e-02, tolerance: 1.352e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.284e-02, tolerance: 1.259e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.807e-02, tolerance: 8.388e

[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END ...................................lasso__alpha=0.1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END .....................................lasso__alpha=1; total time=   0.0s
[CV] END ...................

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df is your training DataFrame
# Assuming test_df is your testing DataFrame

# Extract features and target variable from training data
X_train = train_df[['PRON']]  # Only the 'PRON' column for features
y_train = train_df['score']   # Target variable

# Extract features, target variable, and names from testing data
X_test = test_df[['PRON']]
y_test = test_df['score']
names_test = test_df['name']

# Define the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_pron', CountVectorizer(), 'PRON')  # Only 'PRON' column
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR())
])

# Define the parameter grid to be searched
param_grid = {
    'svr__C': [0.1, 1, 10, 100],            # Example values
    'svr__kernel': ['linear', 'rbf'],       # Example values
    'svr__gamma': ['scale', 'auto']         # Example values
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=7, scoring='neg_mean_absolute_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best Mean Absolute Error:", -grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print predicted, actual values, and names
for actual, predicted, name in zip(y_test, y_pred, names_test):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Fitting 7 folds for each of 16 candidates, totalling 112 fits
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ...svr__C=0.1, svr__gamma=scale, svr__kernel=linear; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, svr__kernel=rbf; total time=   0.0s
[CV] END ......svr__C=0.1, svr__gamma=scale, sv

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df is your training DataFrame
# Assuming test_df is your testing DataFrame

# Extract features and target variable from training data
X_train = train_df[['AUX', 'NUM', 'ADV']]  # Columns for features
y_train = train_df['score']                # Target variable

# Extract features, target variable, and names from testing data
X_test = test_df[['AUX', 'NUM', 'ADV']]
y_test = test_df['score']
names_test = test_df['name']

# Define the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM'),
        ('vect_adv', CountVectorizer(), 'ADV')
    ]
)

# Define the pipeline with the column transformer and Elastic Net
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('elastic_net', ElasticNet())
])

# Define the parameter grid to be searched
param_grid = {
    'elastic_net__alpha': [0.01, 0.1, 1, 10, 100],  # Example values
    'elastic_net__l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0]  # Example values
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=7, scoring='neg_mean_absolute_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best Mean Absolute Error:", -grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print predicted, actual values, and names
for actual, predicted, name in zip(y_test, y_pred, names_test):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Fitting 7 folds for each of 25 candidates, totalling 175 fits
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.255e-02, tolerance: 1.352e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.312e-02, tolerance: 1.259e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.319e-02, tolerance: 1.352e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.278e-02, tolerance: 1.259e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.842e-02, tolerance: 8.388e

[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END .elastic_net__alpha=0.01, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END elastic_net__alpha=0.01, elastic_net__l1_ratio=0.75; total time=   0.0s


...

[CV] END ....elastic_net__alpha=1, elastic_net__l1_ratio=0.0; total time=   0.1s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ...elastic_net__alpha=1, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ....elastic_net__alpha=1, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END ....elastic_net__alpha=1, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END ....elastic_net__alpha=1, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END ....elastic_net__alpha=1, elastic_net__l1_ratio=0.5; total time=   0.0s
[CV] END ....elastic_net__al

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.323e+00, tolerance: 1.352e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.082e+00, tolerance: 1.259e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ...elastic_net__alpha=10, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=10, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ..elastic_net__alpha=10, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ..elastic_net__alpha=10, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ..elastic_net__alpha=10, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ..elastic_net__alpha=10, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END ..elastic_net__alph

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.103e+01, tolerance: 1.352e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.973e+01, tolerance: 1.259e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END ..elastic_net__alpha=100, elastic_net__l1_ratio=0.0; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha=100, elastic_net__l1_ratio=0.25; total time=   0.0s
[CV] END .elastic_net__alpha

__________________________

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['clean_text'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the SVC model
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)

                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10


Confused Samples:
{'Name': 'bundy', 'Clean Text': ' yes thats true i grew up in a wonderful home with two dedicated and loving parents one of five brothers and sisters but as a young a a young boy and i mean the boy of uh twelve or thirteen certainly that i encountered outside the home again in um the local grocery store the local drugstore the soft core pornography what people call soft core um but as i think i i explained to you last night doctor dobson in an anecdote that as young boys do we explore the the back roads and sideways and byways ', 'True Label': 'psychopathic', 'Predicted Label': 'not psychopathic'}
{'Name': 'kemper', 'Clean

# Classification; Pronouns; SVC

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['pronouns'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the SVC model
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)


                  precision    recall  f1-score   support

not psychopathic       1.00      0.75      0.86         4
    psychopathic       0.86      1.00      0.92         6

        accuracy                           0.90        10
       macro avg       0.93      0.88      0.89        10
    weighted avg       0.91      0.90      0.90        10


Confused Samples:
{'Name': 'gacy', 'Clean Text': ' well thats hogwash yeah thats strictly hogwash im constantly told that it that i im misquoted constantly saying that i i said this and i said that and its not happened right ive always felt that my attorneys were handling it and and they said to keep a low profile and stay away from it and uh in light of all that has been used against me in the media theyve theyve created this fantasy monster image and its been going on for the last twelve years and ive ever had no comment and i had no uh need ', 'True Label': 'not psychopathic', 'Predicted Label': 'psychopathic'}


## Classification: Pronouns; Logistic Regression

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['pronouns'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)


                  precision    recall  f1-score   support

not psychopathic       1.00      0.75      0.86         4
    psychopathic       0.86      1.00      0.92         6

        accuracy                           0.90        10
       macro avg       0.93      0.88      0.89        10
    weighted avg       0.91      0.90      0.90        10


Confused Samples:
{'Name': 'gacy', 'Clean Text': ' well thats hogwash yeah thats strictly hogwash im constantly told that it that i im misquoted constantly saying that i i said this and i said that and its not happened right ive always felt that my attorneys were handling it and and they said to keep a low profile and stay away from it and uh in light of all that has been used against me in the media theyve theyve created this fantasy monster image and its been going on for the last twelve years and ive ever had no comment and i had no uh need ', 'True Label': 'not psychopathic', 'Predicted Label': 'psychopathic'}


In [ ]:
df.columns

Index(['name', 'score', 'text', 'clean_text', 'binary', 'trinary', 'group',
       'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       'SPACE'],
      dtype='object')

0    was was s had would was do be is m are can be ...
1                                was was be is is m be
2    do would re might would is would is will is wa...
3    did was did ve been was did did were can was a...
4    was was was were were was do been been be is i...
5    s m m ve were has been ve ve been ve had were ...
6    has been would be is did did did m is was was ...
7    ca do was should have been do is do do was was...
8    would be may is can have am was did was ll was...
9    did did was being was do was could was did mus...
Name: AUX, dtype: object

## get the loocv perofrmance of a feature

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut

def perform_leave_one_out_cv(column_name, data):
    """
    Perform leave-one-out cross-validation on a specified column of a DataFrame.

    Args:
    column_name (str): The name of the column to be used as features.
    data (DataFrame): The DataFrame containing the dataset.

    Returns:
    None: Prints the average Mean Squared Error and R-squared values.
    """

    # Extract features (bag of words) and target variable
    X = data[column_name]
    y = data['score']
    names = data['name']  # Assuming there is a 'name' column in the dataset

    # Initialize variables to store evaluation metrics
    mse_values = []
    r2_values = []

    # Initialize lists to store predicted and actual values, and names
    predicted_values = []
    actual_values = []
    fold_names = []

    # Vectorize the text data using CountVectorizer
    vectorizer = CountVectorizer()

    # Perform leave-one-out cross-validation
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        name = names.iloc[test_index].values[0]  # Get the name for this fold

        # Check if X_train is empty or contains only stop words
        if not X_train.any():
            continue  # Skip to the next iteration if X_train is empty
        else:
            # Vectorize the training and test data
            X_train_vectorized = vectorizer.fit_transform(X_train)
            X_test_vectorized = vectorizer.transform(X_test)

            # Initialize and fit the Support Vector Regression model
            svr = SVR(kernel='linear')
            svr.fit(X_train_vectorized, y_train)

            # Make predictions on the test point
            y_pred = svr.predict(X_test_vectorized)

            # Calculate Mean Squared Error for this fold
            mse = mean_squared_error(y_test, y_pred)

            # Append MSE to the list
            mse_values.append(mse)

            # Append the real and predicted values
            actual_values.append(y_test.values[0])
            predicted_values.append(y_pred[0])

            # Append the name for reference
            fold_names.append(name)

    # Calculate the average MSE over all folds
    average_mse = sum(mse_values) / len(mse_values)

    # Calculate R-squared using the real and predicted values
    r2 = r2_score(actual_values, predicted_values)

    # Print average evaluation metrics
    print(f"Column: {column_name}")
    print(f"Average Mean Squared Error (MSE): {average_mse}")
    print(f"R-squared (R2): {r2}")

    # Print predicted and actual values along with names
    for i in range(len(fold_names)):
        print(f"Fold {i + 1} - Name: {fold_names[i]}, Actual: {actual_values[i]}, Predicted: {predicted_values[i]}")

# Example usage of the function
# df = pd.read_csv('your_data.csv')  # Load your dataset
# perform_leave_one_out_cv('AUX', df)


# Feature Selection: SVR over each fetures alone; leave one out cross validation to get R squared and MSE

In [ ]:
u = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       'SPACE']

In [ ]:
for column in u:
  try:
    perform_leave_one_out_cv(column, data)
  except:
    continue

Column: ADJ
Average Mean Squared Error (MSE): 44.47511151540789
R-squared (R2): -0.10903588343389803
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 31.753475755176844
Fold 2 - Name: carl, Actual: 36.0, Predicted: 32.63615814343991
Fold 3 - Name: bundy, Actual: 34.0, Predicted: 28.84432202031456
Fold 4 - Name: bernardo, Actual: 35.0, Predicted: 28.995357988048102
Fold 5 - Name: dahmer, Actual: 23.0, Predicted: 31.87257122087299
Fold 6 - Name: gacy, Actual: 27.0, Predicted: 30.602268530282487
Fold 7 - Name: kemper, Actual: 26.0, Predicted: 31.80748874887369
Fold 8 - Name: marsh, Actual: 34.5, Predicted: 34.117142351577186
Fold 9 - Name: mitchell, Actual: 34.0, Predicted: 35.23689486344937
Fold 10 - Name: ridgeway, Actual: 19.0, Predicted: 32.2548672868332
Column: ADP
Average Mean Squared Error (MSE): 174.69010241322974
R-squared (R2): -3.35609007950202
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 28.69056619390431
Fold 2 - Name: carl, Actual: 36.0, Predicted: 47.427048770855066
Fold 3 

### function where you can combine features to feed into svr model

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def perform_leave_one_out_cv(data, feature_columns, target_column='score', name_column='name'):
    """
    Perform leave-one-out cross-validation using specified feature columns.

    Args:
    data (DataFrame): The DataFrame containing the dataset.
    feature_columns (list of str): List of column names to be used as features.
    target_column (str): Name of the target column.
    name_column (str): Name of the column containing names for reference.

    Returns:
    None: Prints the average Mean Squared Error, R-squared, and Adjusted R-squared values.
    """

    # Extract features and target variable
    X = data[feature_columns]
    y = data[target_column]
    names = data[name_column]

    # Initialize variables to store evaluation metrics
    mse_values = []
    r2_values = []
    adjusted_r2_values = []

    # Initialize lists to store predicted and actual values, and names
    predicted_values = []
    actual_values = []
    fold_names = []

    # Define the column transformer dynamically
    transformers = [(f'vect_{col}', CountVectorizer(), col) for col in feature_columns]
    column_transformer = ColumnTransformer(transformers=transformers)

    # Define the pipeline with the column transformer and SVR
    pipeline = Pipeline([
        ('transformer', column_transformer),
        ('svr', SVR(kernel='linear'))
    ])

    # Perform leave-one-out cross-validation
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        name = names.iloc[test_index].values[0]

        # Check if X_train is empty
        if X_train.isnull().all().all():
            continue
        else:
            # Fit the pipeline on the training data
            pipeline.fit(X_train, y_train)

            # Make predictions on the test point
            y_pred = pipeline.predict(X_test)

            # Calculate Mean Squared Error for this fold
            mse = mean_squared_error(y_test, y_pred)

            # Append MSE to the list
            mse_values.append(mse)

            # Append the real and predicted values
            actual_values.append(y_test.values[0])
            predicted_values.append(y_pred[0])

            # Append the name for reference
            fold_names.append(name)

    # Calculate the average MSE over all folds
    average_mse = sum(mse_values) / len(mse_values)

    # Calculate R-squared using the real and predicted values
    r2 = r2_score(actual_values, predicted_values)

    # Calculate Adjusted R-squared
    n = len(X)
    k = len(feature_columns)
    adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

    # Print average evaluation metrics
    print(f"Columns: {', '.join(feature_columns)}")
    print(f"Average Mean Squared Error (MSE): {average_mse}")
    print(f"R-squared (R2): {r2}")
    print(f"Adjusted R-squared: {adjusted_r2}")

    # Print predicted and actual values along with names
    for i in range(len(fold_names)):
        print(f"Fold {i + 1} - Name: {fold_names[i]}, Actual: {actual_values[i]}, Predicted: {predicted_values[i]}")


# Example usage
# df = pd.read_csv('your_data.csv')
# feature_cols = ['PRON', 'AUX', 'NUM', 'VERB', 'ADV']
# perform_leave_one_out_cv(df, feature_cols)


### Try out all combos of best performing features: leave one out

In [ ]:
all_combinations = [
    ['PRON'],
    ['AUX'],
    ['NUM'],
    ['VERB'],
    ['ADV'],
    ['PRON', 'AUX'],
    ['PRON', 'NUM'],
    ['PRON', 'VERB'],
    ['PRON', 'ADV'],
    ['AUX', 'NUM'],
    ['AUX', 'VERB'],
    ['AUX', 'ADV'],
    ['NUM', 'VERB'],
    ['NUM', 'ADV'],
    ['VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM'],
    ['PRON', 'AUX', 'VERB'],
    ['PRON', 'AUX', 'ADV'],
    ['PRON', 'NUM', 'VERB'],
    ['PRON', 'NUM', 'ADV'],
    ['PRON', 'VERB', 'ADV'],
    ['AUX', 'NUM', 'VERB'],
    ['AUX', 'NUM', 'ADV'],
    ['AUX', 'VERB', 'ADV'],
    ['NUM', 'VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM', 'VERB'],
    ['PRON', 'AUX', 'NUM', 'ADV'],
    ['PRON', 'AUX', 'VERB', 'ADV'],
    ['PRON', 'NUM', 'VERB', 'ADV'],
    ['AUX', 'NUM', 'VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM', 'VERB', 'ADV']
]


In [ ]:
df = copy
for feature_col in all_combinations:
  perform_leave_one_out_cv(df, feature_col)

Columns: PRON
Average Mean Squared Error (MSE): 15.302508671977503
R-squared (R2): 0.6184150945208526
Adjusted R-squared: 0.5707169813359592
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 34.152102086814864
Fold 2 - Name: carl, Actual: 36.0, Predicted: 40.87588530146328
Fold 3 - Name: bundy, Actual: 34.0, Predicted: 33.49653126332478
Fold 4 - Name: bernardo, Actual: 35.0, Predicted: 31.55997695925472
Fold 5 - Name: dahmer, Actual: 23.0, Predicted: 27.272821023322635
Fold 6 - Name: gacy, Actual: 27.0, Predicted: 31.569874686321867
Fold 7 - Name: kemper, Actual: 26.0, Predicted: 23.82139706929955
Fold 8 - Name: marsh, Actual: 34.5, Predicted: 33.05594696612617
Fold 9 - Name: mitchell, Actual: 34.0, Predicted: 35.1597503456635
Fold 10 - Name: ridgeway, Actual: 19.0, Predicted: 24.970620156185614
Columns: AUX
Average Mean Squared Error (MSE): 10.698771030567853
R-squared (R2): 0.7332143624320715
Adjusted R-squared: 0.6998661577360804
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 36.422656

## Best Performing Combo ['AUX', 'NUM'] ----> 0.5 TTS (IS THIS A METHODOLOGICAL ERROR BECAUSE I SELECTED THESE BY TRAINING LOOCV AND NOW IM DOING TTS?)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset into a DataFrame
data = df

# Extract features and target variable using only 'AUX' and 'NUM' columns
X = data[['AUX', 'NUM']]
y = data['score']
names = data['name']  # Assuming you have a 'name' column in your dataset

# Split data into training and test sets (50% train, 50% test)
X_train, X_test, y_train, y_test, names_train, names_test = train_test_split(
    X, y, names, test_size=0.5, random_state=42)

# Define the column transformer for 'AUX' and 'NUM' columns
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM')
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR(kernel='linear'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Columns: AUX, NUM")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Print predicted, actual values, and names
for i in range(len(names_test)):
    print(f"Name: {names_test.iloc[i]}, Actual: {y_test.iloc[i]}, Predicted: {y_pred[i]}")


Columns: AUX, NUM
Mean Squared Error (MSE): 6.536453906553305
R-squared (R2): 0.6319564241805571
Name: mitchell, Actual: 34.0, Predicted: 35.21556397675986
Name: carl, Actual: 36.0, Predicted: 35.22698239904055
Name: gacy, Actual: 27.0, Predicted: 24.78310547878193
Name: wayne, Actual: 40.0, Predicted: 35.45926996279117
Name: marsh, Actual: 34.5, Predicted: 32.247386639753735


## Best Performing Combo ['AUX', 'NUM', 'ADV'] ----> 0.5 TTS

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset into a DataFrame
data = df

# Extract features and target variable using 'AUX', 'NUM', and 'ADV' columns
X = data[['AUX', 'NUM', 'ADV']]  # Include 'ADV' column
y = data['score']
names = data['name']  # Assuming you have a 'name' column in your dataset

# Split data into training and test sets (50% train, 50% test)
X_train, X_test, y_train, y_test, names_train, names_test = train_test_split(
    X, y, names, test_size=0.5, random_state=42)

# Define the column transformer for 'AUX', 'NUM', and 'ADV' columns
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM'),
        ('vect_adv', CountVectorizer(), 'ADV')  # Add transformer for 'ADV' column
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR(kernel='linear'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Columns: AUX, NUM, ADV")  # Update column list in print statement
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Print predicted, actual values, and names
for i in range(len(names_test)):
    print(f"Name: {names_test.iloc[i]}, Actual: {y_test.iloc[i]}, Predicted: {y_pred[i]}")


Columns: AUX, NUM, ADV
Mean Squared Error (MSE): 7.05240514699544
R-squared (R2): 0.6029051155971036
Name: mitchell, Actual: 34.0, Predicted: 34.64903157121174
Name: carl, Actual: 36.0, Predicted: 35.12426432865506
Name: gacy, Actual: 27.0, Predicted: 25.305870682559252
Name: wayne, Actual: 40.0, Predicted: 35.21130939226002
Name: marsh, Actual: 34.5, Predicted: 31.62384998520378


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset into a DataFrame
data = df

# Extract features and target variable using 'AUX', 'NUM', and 'ADV' columns
X = data[['AUX']]  # Include 'ADV' column
y = data['score']
names = data['name']  # Assuming you have a 'name' column in your dataset

# Split data into training and test sets (50% train, 50% test)
X_train, X_test, y_train, y_test, names_train, names_test = train_test_split(
    X, y, names, test_size=0.5, random_state=42)

# Define the column transformer for 'AUX', 'NUM', and 'ADV' columns
column_transformer = ColumnTransformer(
    transformers=[

        ('vect_aux', CountVectorizer(), 'AUX')  # Add transformer for 'ADV' column
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR(kernel='linear'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Columns: AUX, NUM, ADV")  # Update column list in print statement
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Print predicted, actual values, and names
for i in range(len(names_test)):
    print(f"Name: {names_test.iloc[i]}, Actual: {y_test.iloc[i]}, Predicted: {y_pred[i]}")


Columns: AUX, NUM, ADV
Mean Squared Error (MSE): 6.44881825325989
R-squared (R2): 0.6368908641182494
Name: mitchell, Actual: 34.0, Predicted: 35.49899563470074
Name: carl, Actual: 36.0, Predicted: 35.373676882259915
Name: gacy, Actual: 27.0, Predicted: 24.868857742582758
Name: wayne, Actual: 40.0, Predicted: 35.56054658276651
Name: marsh, Actual: 34.5, Predicted: 32.18606207073388
